In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/nmwsharp/diffusion-net.git

fatal: destination path 'diffusion-net' already exists and is not an empty directory.


In [ ]:
!cp -r /content/diffusion-net/src/diffusion_net/ /content/drive/MyDrive

In [ ]:
!pip install  trimesh plyfile polyscope potpourri3d python-utils robust-laplacian threadpoolctl tqdm typing-extensions joblib

In [ ]:
import os
import sys

sys.path.append("/content/drive/MyDrive")
import diffusion_net

# Data augmentation

In [ ]:
import trimesh
import numpy as np
import os
import json
import random


def random_rotation(mesh):
    euler_angles = [random.uniform(0, 360) for _ in range(3)]  # Random rotations in degrees
    rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(euler_angles[0]), [1, 0, 0])
    rotation_matrix = np.dot(rotation_matrix, trimesh.transformations.rotation_matrix(np.radians(euler_angles[1]), [0, 1, 0]))
    rotation_matrix = np.dot(rotation_matrix, trimesh.transformations.rotation_matrix(np.radians(euler_angles[2]), [0, 0, 1]))
    mesh.apply_transform(rotation_matrix)
    return mesh

def random_scaling(mesh, scale_range=(0.8, 1.2)):
    scale_factor = random.uniform(scale_range[0], scale_range[1])
    mesh.apply_scale(scale_factor)
    return mesh

def random_translation(mesh, translation_range=(-0.1, 0.1)):
    translation = [random.uniform(translation_range[0], translation_range[1]) for _ in range(3)]
    mesh.apply_translation(translation)
    return mesh




def data_augmentation(mesh_dirpath,label_dirpath,output_dir):

        # Define the number of augmented samples to generate
        num_augmented_samples = 3
        # Create a directory to store augmented samples
        os.makedirs(output_dir, exist_ok=True)

        # Apply augmentations and save augmented samples
        for fname in os.listdir(mesh_dirpath):

        # Load the original mesh from .obj and .json files

            original_mesh = trimesh.load_mesh(os.path.join(mesh_dirpath, fname))
            label_path = os.path.join(label_dirpath,os.path.splitext(fname)[0]+".json")

            with open(label_path, 'r') as json_file:
                      original_label = json.load(json_file)
            for i in range(num_augmented_samples):
                    augmented_mesh = original_mesh.copy()

                    # Apply geometric transformations to mesh and metadata
                    augmented_mesh = random_rotation(augmented_mesh)
                    augmented_mesh = random_scaling(augmented_mesh)
                    augmented_mesh = random_translation(augmented_mesh)

                    # Save the augmented mesh
                    output_path = os.path.join(output_dir, f'augmented_{os.path.splitext(fname)[0]+"_"+str(i)}.obj')
                    augmented_mesh.export(output_path)

                    # Save the augmented label

                    label_path_ = os.path.join(output_dir, f'augmented_{os.path.splitext(fname)[0]+"_"+str(i)}.json')
                    with open(label_path_, 'w') as json_file:
                        json.dump(original_label, json_file)

        print(f'{num_augmented_samples * len( os.listdir(mesh_dirpath))} augmented samples generated and saved in {output_dir}')


In [ ]:
import matplotlib.pyplot as plt

mesh_dirpath = "/content/drive/MyDrive/dataset-arcades/dataset/obj_files/"
label_dirpath = "/content/drive/MyDrive/dataset-arcades/dataset/G_Truth_Files/"
output_dir = "/content/drive/MyDrive/dataset-arcades/test"
data_augmentation(mesh_dirpath,label_dirpath,output_dir)


mesh = trimesh.load_mesh('/content/drive/MyDrive/dataset-arcades/test/augmented_4_0.obj')

scene = mesh.scene()
scene.show()

57 augmented samples generated and saved in /content/drive/MyDrive/dataset-arcades/test


# Loading and Preparing the dataset

In [ ]:
import shutil
import os
import sys
import random
import numpy as np
import json

import torch
from torch.utils.data import Dataset

import potpourri3d as pp3d

import diffusion_net
from diffusion_net.utils import toNP


class Seg_Dataset(Dataset):

    def __init__(self, root_dir, all_in_one_file, k_eig=128):

        self.all_in_one_file = all_in_one_file  # bool
        self.k_eig = k_eig
        self.root_dir = root_dir
        self.n_class = 15

        # store in memory
        self.verts_list = []
        self.faces_list = []
        self.labels_list = []

        # Get all the files
        mesh_files = []
        label_files = []

        # all_in_one_file split
        if self.all_in_one_file:

            mesh_dirpath = os.path.join(self.root_dir, "obj_files")
            label_dirpath = os.path.join(self.root_dir,"G_Truth_Files")
            for fname in os.listdir(mesh_dirpath):
                mesh_fullpath = os.path.join(mesh_dirpath, fname)
                label_fullpath = os.path.join(label_dirpath,os.path.splitext(fname)[0]+".json")
                mesh_files.append(mesh_fullpath)
                label_files.append(label_fullpath)


        else:

            for fname in os.listdir(self.root_dir):
                if fname.endswith(".obj"):
                  mesh_fullpath = os.path.join(self.root_dir, fname)
                  label_fullpath = os.path.join(self.root_dir,os.path.splitext(fname)[0]+".json")
                  mesh_files.append(mesh_fullpath)
                  label_files.append(label_fullpath)

        print("loading {} meshes".format(len(mesh_files)))

        # Load the actual files
        for iFile in range(len(mesh_files)):

            print("loading mesh " + str(mesh_files[iFile]))

            verts, faces = pp3d.read_mesh(mesh_files[iFile])

            with open(label_files[iFile], 'r') as json_file:
                data = json.load(json_file)
            face_labels = data["face_labels"]
            labels = np.array(face_labels).astype(int)

            # to torch
            verts = torch.tensor(np.ascontiguousarray(verts)).float()
            faces = torch.tensor(np.ascontiguousarray(faces))
            labels = torch.tensor(np.ascontiguousarray(labels))

            # center and unit scale
            verts = diffusion_net.geometry.normalize_positions(verts)

            self.verts_list.append(verts)
            self.faces_list.append(faces)
            self.labels_list.append(labels)

        for ind, labels in enumerate(self.labels_list):
            self.labels_list[ind] = labels

        # Precompute operators
        self.frames_list, self.massvec_list, self.L_list, self.evals_list, self.evecs_list, self.gradX_list, self.gradY_list = diffusion_net.geometry.get_all_operators(self.verts_list, self.faces_list, k_eig=self.k_eig)

    def __len__(self):
        return len(self.verts_list)

    def __getitem__(self, idx):
        return self.verts_list[idx], self.faces_list[idx], self.frames_list[idx], self.massvec_list[idx], self.L_list[idx], self.evals_list[idx], self.evecs_list[idx], self.gradX_list[idx], self.gradY_list[idx], self.labels_list[idx]

In [ ]:
train_dir = "/content/drive/MyDrive/dataset-arcades/test"
#train_dir = "/content/drive/MyDrive/archesDataset/train"
k_eig = 256
train_dataset = Seg_Dataset(train_dir, False, k_eig)

loading 57 meshes
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_10_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_10_Upper_2.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_11_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_11_Upper_2.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_12_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_12_Upper_2.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_9_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_9_Upper_2.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_22_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_22_Upper_2.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_23_Upper_1.obj
loading mesh /content/drive/MyDrive/dataset-arcades/test/augmented_23_Upper_2.obj


In [ ]:
test_dir = "/content/drive/MyDrive/dataset-arcades/dataset"
#test_dir = "/content/drive/MyDrive/archesDataset/test"
k_eig = 256
test_dataset = Seg_Dataset(test_dir, True, k_eig)

loading 19 meshes
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/10_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/11_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/12_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/9_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/22_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/23_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/20_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/21_Upper.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/19.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/8.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/6.obj
loading mesh /content/drive/MyDrive/dataset-arcades/dataset/obj_files/15.obj
loading mesh 

# Training the model

In [ ]:
import os
import sys
import argparse
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

import diffusion_net



# system things
device = torch.device('cuda:0')
dtype = torch.float32

# problem/dataset things
n_class = 15

# model
input_features = 'xyz'

# training settings
n_epoch = 600
lr = 1e-3
decay_every = 600
decay_rate = 0.5
augment_random_rotate = (input_features == 'xyz')



# === Load datasets

# Load the train dataset
train_loader = DataLoader(train_dataset, batch_size=None, shuffle=True)

# Load the test dataset
test_loader = DataLoader(test_dataset, batch_size=None)




# === Create the model

C_in={'xyz':3, 'hks':16}[input_features] # dimension of input features

model = diffusion_net.layers.DiffusionNet(C_in=C_in,
                                          C_out=n_class,
                                          C_width=256,
                                          N_block=4,
                                          last_activation=lambda x : torch.nn.functional.log_softmax(x,dim=-1),
                                          outputs_at='faces',
                                          dropout=True)


model = model.to(device)



# === Optimize
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def train_epoch(epoch):

    # Implement lr decay
    if epoch > 0 and epoch % decay_every == 0:
        global lr
        lr *= decay_rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


    # Set model to 'train' mode
    model.train()
    optimizer.zero_grad()

    correct = 0
    total_num = 0
    for data in tqdm(train_loader):

        # Get data
        verts, faces, frames, mass, L, evals, evecs, gradX, gradY, labels = data

        # Move to device
        verts = verts.to(device)
        faces = faces.to(device)
        frames = frames.to(device)
        mass = mass.to(device)
        L = L.to(device)
        evals = evals.to(device)
        evecs = evecs.to(device)
        gradX = gradX.to(device)
        gradY = gradY.to(device)
        labels = labels.to(device)

        # Randomly rotate positions
        if augment_random_rotate:
            verts = diffusion_net.utils.random_rotate_points(verts)

        # Construct features
        if input_features == 'xyz':
            features = verts
        elif input_features == 'hks':
            features = diffusion_net.geometry.compute_hks_autoscale(evals, evecs, 16)

        # Apply the model
        preds = model(features, mass, L=L, evals=evals, evecs=evecs, gradX=gradX, gradY=gradY, faces=faces)

        # Evaluate loss

        loss = torch.nn.functional.nll_loss(preds, labels)
        loss.backward()

        # track accuracy
        pred_labels = torch.max(preds, dim=1).indices
        this_correct = pred_labels.eq(labels).sum().item()
        this_num = labels.shape[0]
        correct += this_correct
        total_num += this_num

        # Step the optimizer
        optimizer.step()
        optimizer.zero_grad()

    train_acc = correct / total_num
    return train_acc , loss


# Do an evaluation pass on the test dataset


def test():

    model.eval()

    correct = 0
    total_num = 0
    with torch.no_grad():

        for data in tqdm(test_loader):

            # Get data
            verts, faces, frames, mass, L, evals, evecs, gradX, gradY, labels = data

            # Move to device
            verts = verts.to(device)
            faces = faces.to(device)
            frames = frames.to(device)
            mass = mass.to(device)
            L = L.to(device)
            evals = evals.to(device)
            evecs = evecs.to(device)
            gradX = gradX.to(device)
            gradY = gradY.to(device)
            labels = labels.to(device)

            # Construct features
            if input_features == 'xyz':
                features = verts
            elif input_features == 'hks':
                features = diffusion_net.geometry.compute_hks_autoscale(evals, evecs, 16)

            # Apply the model
            preds = model(features, mass, L=L, evals=evals, evecs=evecs, gradX=gradX, gradY=gradY, faces=faces)

            loss = torch.nn.functional.nll_loss(preds, labels)

            # track accuracy
            pred_labels = torch.max(preds, dim=1).indices
            this_correct = pred_labels.eq(labels).sum().item()
            this_num = labels.shape[0]
            correct += this_correct
            total_num += this_num

    test_acc = correct / total_num
    return test_acc , loss


print("Training...")
train_accs = []
train_losses = []
test_accs = []
test_losses = []

for epoch in range(n_epoch):
        train_acc , train_loss = train_epoch(epoch)
        train_accs.append(train_acc)
        train_losses.append(train_loss)
        test_acc , test_loss = test()
        test_accs.append(test_acc)
        test_losses.append(test_loss)
        print("Epoch {} - Train overall: {:06.3f}% -  Test overall: {:06.3f}% ".format(epoch, 100*train_acc, 100*test_acc))



Training...


100%|██████████| 19/19 [00:00<00:00, 34.49it/s]


Epoch 0 - Train overall: 32.869% -  Test overall: 33.304% 


100%|██████████| 19/19 [00:00<00:00, 35.13it/s]


Epoch 1 - Train overall: 33.248% -  Test overall: 33.304% 


...

Epoch 597 - Train overall: 83.044% -  Test overall: 82.215% 


100%|██████████| 19/19 [00:00<00:00, 33.21it/s]


Epoch 598 - Train overall: 82.968% -  Test overall: 83.622% 


100%|██████████| 19/19 [00:00<00:00, 30.52it/s]

Epoch 599 - Train overall: 82.967% -  Test overall: 84.223% 


# Visualizing the DiffusionNets Performance

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

epochs = list(range(1, 601))

fig = go.Figure()

# Ajouter la courbe d'entraînement
fig.add_trace(go.Scatter(x=epochs, y=train_accs, mode='lines', name='Train Accuracy'))

# Ajouter la courbe de test
fig.add_trace(go.Scatter(x=epochs, y=test_accs, mode='lines', name='Test Accuracy'))

# Mise en forme du graphique
fig.update_layout(title='Accuracy Over 600 Epochs', xaxis_title='Epoch', yaxis_title='Accuracy')

fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import torch
import numpy as np

epochs = list(range(1, 601))

# Assuming you have train_losses and test_losses as PyTorch tensors on GPU
train_losses_cpu = [tensor.cpu().item() for tensor in train_losses]
test_losses_cpu = [tensor.cpu().item() for tensor in test_losses]

fig = go.Figure()

# Ajouter la courbe d'entraînement
fig.add_trace(go.Scatter(x=epochs, y=train_losses_cpu, mode='lines', name='Train Loss'))

# Ajouter la courbe de test
fig.add_trace(go.Scatter(x=epochs, y=test_losses_cpu, mode='lines', name='Test Loss'))

# Mise en forme du graphique
fig.update_layout(title='Loss Over 600 Epochs', xaxis_title='Epoch', yaxis_title='Loss')

fig.show()


# Test on a 3d object

In [ ]:
import trimesh
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


# Load the segmented mesh
mesh_path="/content/drive/MyDrive/dataset-arcades/test/augmented_2_0.obj"
verts, faces = pp3d.read_mesh(mesh_path)

verts = torch.tensor(np.ascontiguousarray(verts)).float()
faces = torch.tensor(np.ascontiguousarray(faces))

# center and unit scale
verts = diffusion_net.geometry.normalize_positions(verts)

frames, mass, L, evals, evecs, gradX, gradY = diffusion_net.geometry.get_operators(verts,faces, k_eig=1024)

# Move to device
verts = verts.to(device)
faces = faces.to(device)
frames = frames.to(device)
mass = mass.to(device)
L = L.to(device)
evals = evals.to(device)
evecs = evecs.to(device)
gradX = gradX.to(device)
gradY = gradY.to(device)

features = verts

# Apply the model
preds = model(features, mass, L=L, evals=evals, evecs=evecs, gradX=gradX, gradY=gradY, faces=faces)


# track accuracy
pred_labels = torch.max(preds, dim=1).indices




vertices, faces = pp3d.read_mesh(mesh_path)


# Assuming you have predicted labels for each face
predicted_labels = pred_labels.cpu().numpy()  # List of predicted labels for each face (0 to 7)

# Define colors for each class
class_colors = [
    (1.0, 0.0, 0.0),   # Class 0 - Red
    (0.0, 1.0, 0.0),   # Class 1 - Green
    (0.0, 0.0, 1.0),   # Class 2 - Blue
    (1.0, 1.0, 0.0),   # Class 3 - Yellow
    (1.0, 0.0, 1.0),   # Class 4 - Magenta
    (0.0, 1.0, 1.0),   # Class 5 - Cyan
    (0.5, 0.5, 0.5),   # Class 6 - Gray
    (0.0, 0.0, 0.0),   # Class 7 - Black
    (0.5, 1.0, 1.0),   # Class 8 - Light Cyan
    (1.0, 0.5, 0.0),   # Class 9 - Orange
    (0.5, 1.0, 0.0),   # Class 10 - Lime
    (1.0, 0.5, 0.5),   # Class 11 - Light Pink
    (0.0, 0.5, 1.0),   # Class 12 - Light Blue
    (0.5, 0.0, 1.0),   # Class 13 - Purple
    (0.5, 0.5, 0.0),   # Class 14 - Olive
]


# Create a Plotly figure
fig = go.Figure()

# Loop through each face and plot it with the corresponding color
for face, label in zip(faces, predicted_labels):
    vertex_indices = face
    face_vertices = vertices[vertex_indices]

    # Get the color based on the label
    face_color = class_colors[label]

    # Create a mesh3d trace for each face
    mesh_trace = go.Mesh3d(x=face_vertices[:, 0], y=face_vertices[:, 1], z=face_vertices[:, 2],
                           color="rgba({}, {}, {}, 1)".format(*[int(255 * c) for c in face_color]),
                           opacity=0.5)

    # Add the trace to the figure
    fig.add_trace(mesh_trace)

# Set layout properties
fig.update_layout(scene=dict(aspectmode="data"))
fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"))

# Show the interactive Plotly figure
fig.show()


Output hidden; open in https://colab.research.google.com to view.

# Saving the trained model

In [ ]:
model_path = '/content/drive/MyDrive/2.0arches_trained_model_segmentation.pth'

# Save the model's state dictionary to the specified file
torch.save(model.state_dict(), model_path)

print("Model saved to:", model_path)

Model saved to: /content/drive/MyDrive/2.0arches_trained_model_segmentation.pth
